In [6]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")

model = (AutoModelForCausalLM.from_pretrained
    (
    "microsoft/Phi-3.5-mini-instruct", 
    device_map="auto",
    torch_dtype="auto",
    #attn_implementation = 'eager'
    )
)

rag_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    torch_dtype="auto",
    #attn_implementation = 'eager'
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


In [7]:
from sentence_transformers import SentenceTransformer
import chromadb


db_path="./vektor_DB"

#lädt datenbank lokal runter
client = chromadb.PersistentClient(path=db_path)

#unser emdedding model
embedding_model = SentenceTransformer("sentence-transformers/gtr-t5-large")

#"meinungen" aus der Datenbank laden
collection = client.get_collection("meinungen")

#retrievet daten von unserer Vektorendatenbank
def query_collection(query_text, n_results=4):

    #encode query text (-> unser input)
    query_embedding = embedding_model.encode(query_text)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results,
        include=["documents", "metadatas", "distances"]
    )
    return results

In [8]:
#Funktion, die die Datenbank abfragt und eine Antwort generiert
def query_with_phi3_5(query_text):
    results = query_collection(query_text)
    context = ""
    for i in range(len(results["documents"][0])):
        context += f"Document {i+1}:\n"
        context += f"{results['documents'][0][i]}\n"
        context += f"Party: {results['metadatas'][0][i]['party']}\n"


    prompt = f"""
    You are a political AI assistant.  Based on the following information retrieved from a vector database, answer the user's question.  If the provided data is not relevant to the question, respond with "I do not know".

    User Question: {query_text}

    Retrieved Data:
    {context}

    Your Answer:
    """

    generated_text = rag_pipeline(prompt, max_new_tokens=250)[0]["generated_text"]
    # Extract the answer from the generated text (remove the prompt)
    answer = generated_text[len(prompt):].strip()
    return answer


query = "Was denkt die AFD über Islamunterricht an deutschen Schulen?"
answer = query_with_phi3_5(query)
print(answer)

Die AfD sieht den Islamunterricht an deutschen Schulen kritisch und befürwortet eine sachliche Islamkunde im Ethikunterricht. Sie befürchten, dass der Islamunterricht echte Integration verhindern könnte, da islamische Gemeinschaften keine kirchenähnliche Struktur aufweisen und daher kein „bekenntnisgebundener“ Religionsunterricht zugestanden werden kann. Die Partei fordert, dass Imame und Lehrer*innen in deutscher Sprache und mit einem Zertifikat B2 für die deutsche Sprache des Gemeinsamen Europäischen Referenzrahmens für Sprachen zugelassen werden. Zudem lehnt die AfD die Verleihung des Status als Körperschaft öffentlichen Rechts an islamische Organisationen ab und fordert die Abschaffung der islamtheologischen Lehrstühle an deutschen Universitäten. Sie unterstützt zudem die Gleichberechtigung von Mann und Frau und fordert die Untersagung des Tragens von Burka und Niqab in der Öffentlichkeit.


Document:


In [9]:
from sentence_transformers import SentenceTransformer
import chromadb
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")

model = (AutoModelForCausalLM.from_pretrained
    (
    "microsoft/Phi-3.5-mini-instruct", 
    device_map="auto",
    torch_dtype="auto",
    #attn_implementation = 'eager'
    )
)

rag_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    torch_dtype="auto",
    #attn_implementation = 'eager'
)


db_path="./vektor_DB"

#lädt datenbank lokal runter
client = chromadb.PersistentClient(path=db_path)

#unser emdedding model
embedding_model = SentenceTransformer("sentence-transformers/gtr-t5-large")

#"meinungen" aus der Datenbank laden
collection = client.get_collection("meinungen")

#retrievet daten von unserer Vektorendatenbank
def query_collection(query_text, n_results=4):

    #encode query text (-> unser input)
    query_embedding = embedding_model.encode(query_text)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results,
        include=["documents", "metadatas", "distances"]
    )
    return results

#Funktion, die die Datenbank abfragt und eine Antwort generiert
def query_with_phi3_5(query_text):
    results = query_collection(query_text)
    context = ""
    for i in range(len(results["documents"][0])):
        context += f"Document {i+1}:\n"
        context += f"{results['documents'][0][i]}\n"
        context += f"Party: {results['metadatas'][0][i]['party']}\n"


    prompt = f"""
    You are a political AI assistant.  Based on the following information retrieved from a vector database, answer the user's question.  If the provided data is not relevant to the question, respond with "I do not know".

    User Question: {query_text}

    Retrieved Data:
    {context}

    Your Answer:
    """

    generated_text = rag_pipeline(prompt, max_new_tokens=250)[0]["generated_text"]
    # Extract the answer from the generated text (remove the prompt)
    answer = generated_text[len(prompt):].strip()
    return answer

query = "Was denken die Grünen zum Thema Klimaschutz?"
answer = query_with_phi3_5(query)
print(answer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.
Device set to use cpu


KeyboardInterrupt: 

In [7]:
from sklearn.metrics import accuracy_score, f1_score

y_true = [1]  # Wahre Label (z.B. Ja=1, Nein=0)
y_pred = [1]  # Vorhergesagte Label

accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")


Accuracy: 1.0000
F1 Score: 1.0000


Reference Entities: {'AFD', 'deutschen'}
Candidate Entities: {'AFD', 'deutschen'}
Matching Entities: {'AFD', 'deutschen'}


In [3]:
import spacy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sentence_transformers import SentenceTransformer, util

# Lade das deutsche Spacy-Modell
try:
    nlp = spacy.load("de_core_news_sm")
except OSError:
    import subprocess
    subprocess.run(["python", "-m", "spacy", "download", "de_core_news_sm"])
    nlp = spacy.load("de_core_news_sm")

# Embedding-Modell für semantische Ähnlichkeit
embedding_model = SentenceTransformer("sentence-transformers/gtr-t5-large")

# Referenz- und Kandidatenantworten
reference = "Die AFD lehnt den Islamunterricht an deutschen Schulen ab und betrachtet ihn als problematisch."
candidate = "Die AFD ist kritisch gegenüber dem Islamunterricht in deutschen Schulen."

# Funktion: Berechne semantische Ähnlichkeit
def calculate_semantic_similarity(reference, candidate):
    ref_embedding = embedding_model.encode(reference, convert_to_tensor=True)
    cand_embedding = embedding_model.encode(candidate, convert_to_tensor=True)
    similarity = util.cos_sim(ref_embedding, cand_embedding).item()
    return similarity

# Funktion: Entitäten extrahieren
def extract_entities(text):
    doc = nlp(text)
    return {ent.text for ent in doc.ents}

# Berechnung der Metriken
def evaluate_model(reference, candidate):
    # Debugging: Originaltexte anzeigen
    print(f"Reference: {reference}")
    print(f"Candidate: {candidate}")

    # 1. Accuracy (basierend auf exakter Übereinstimmung)
    accuracy = 1.0 if reference == candidate else 0.0
    print(f"Accuracy (exact match): {accuracy}")

    # 2. Semantische Ähnlichkeit
    semantic_similarity = calculate_semantic_similarity(reference, candidate)
    print(f"Semantic Similarity: {semantic_similarity:.4f}")

    # 3. Precision, Recall, F1-Score (basierend auf Entitäten)
    ref_entities = extract_entities(reference)
    cand_entities = extract_entities(candidate)

    # Debugging: Extrahierte Entitäten anzeigen
    print(f"Reference Entities: {ref_entities}")
    print(f"Candidate Entities: {cand_entities}")

    true_positive = len(ref_entities & cand_entities)
    false_positive = len(cand_entities - ref_entities)
    false_negative = len(ref_entities - cand_entities)

    # Debugging: TP, FP, FN anzeigen
    print(f"True Positives: {true_positive}")
    print(f"False Positives: {false_positive}")
    print(f"False Negatives: {false_negative}")

    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0.0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0.0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    return {
        "Accuracy": accuracy,
        "Average Semantic Similarity": semantic_similarity,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1
    }

# Evaluation durchführen
results = evaluate_model(reference, candidate)

# Ergebnisse ausgeben
for metric, score in results.items():
    print(f"{metric}: {score:.4f}")


Reference: Die AFD lehnt den Islamunterricht an deutschen Schulen ab und betrachtet ihn als problematisch.
Candidate: Die AFD ist kritisch gegenüber dem Islamunterricht in deutschen Schulen.
Accuracy (exact match): 0.0
Semantic Similarity: 0.9409
Reference Entities: {'AFD', 'deutschen'}
Candidate Entities: {'AFD', 'deutschen'}
True Positives: 2
False Positives: 0
False Negatives: 0
Accuracy: 0.0000
Average Semantic Similarity: 0.9409
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000


In [4]:
from sentence_transformers import SentenceTransformer, util
import json

# Lade das Embedding-Modell
similarity_model = SentenceTransformer("sentence-transformers/gtr-t5-large")

# Funktion zur Berechnung der semantischen Ähnlichkeit
def calc_similarity(expected, generated):
    embeddings = similarity_model.encode([expected, generated])
    return util.cos_sim(embeddings[0], embeddings[1]).item()

# Evaluierungsfunktion
def eval_semantics(evaluation_data, threshold=0.7):
    semantic_scores = []
    tp, fp, fn = 0, 0, 0  # Initialisiere die Zähler für Metriken

    for example in evaluation_data:
        query = example["query"]
        expected = example["expected"]
        generated = example["generated"]  # Direkte Nutzung der generierten Antworten

        similarity = calc_similarity(expected, generated)
        semantic_scores.append(similarity)

        # Korrektheit basierend auf Schwellenwert bestimmen
        if similarity > threshold:
            if expected != "I do not know":  # "I do not know" wird als falsch betrachtet
                tp += 1
            else:
                fp += 1
        elif expected != "I do not know":
            fn += 1

    # Berechnung der Metriken
    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

    accuracy = tp / len(evaluation_data)  # Anteil korrekter Antworten
    avg_similarity = sum(semantic_scores) / len(semantic_scores)

    return {
        "accuracy": accuracy,
        "average_similarity": avg_similarity,
        "precision": precision,
        "recall": recall,
        "f1_score": f1_score,
    }

# Beispiel-Datensatz laden
with open("eval.json", "r") as f:
    eval_data = [
    {
        "query": "Was denkt die AFD über Islamunterricht an deutschen Schulen?",
        "expected": "Die AFD lehnt den Islamunterricht an deutschen Schulen ab.",
        "generated": "Die AfD sieht den Islamunterricht an deutschen Schulen kritisch und befürwortet eine sachliche Islamkunde im Ethikunterricht. Sie befürchten, dass der Islamunterricht echte Integration verhindern könnte, da islamische Gemeinschaften keine kirchenähnliche Struktur aufweisen und daher kein „bekenntnisgebundener“ Religionsunterricht zugestanden werden kann. Die Partei fordert, dass Imame und Lehrer*innen in deutscher Sprache und mit einem Zertifikat B2 für die deutsche Sprache des Gemeinsamen Europäischen Referenzrahmens für Sprachen zugelassen werden. Zudem lehnt die AfD die Verleihung des Status als Körperschaft öffentlichen Rechts an islamische Organisationen ab und fordert die Abschaffung der islamtheologischen Lehrstühle an deutschen Universitäten. Sie unterstützt zudem die Gleichberechtigung von Mann und Frau und fordert die Untersagung des Tragens von Burka und Niqab in der Öffentlichkeit."
    },
    {
        "query": "Wie steht die SPD zur Klimapolitik?",
        "expected": "Die SPD setzt sich für eine ambitionierte Klimapolitik ein.",
        "generated": "Die SPD unterstützt eine ambitionierte Klimapolitik."
    }
]


metrics = eval_semantics(eval_data)

# Ergebnisse ausgeben
print(f"Accuracy: {metrics['accuracy'] * 100:.2f}%")
print(f"Average Semantic Similarity: {metrics['average_similarity'] * 100:.2f}%")
print(f"Precision: {metrics['precision'] * 100:.2f}%")
print(f"Recall: {metrics['recall'] * 100:.2f}%")
print(f"F1 Score: {metrics['f1_score'] * 100:.2f}%")


FileNotFoundError: [Errno 2] No such file or directory: 'eval.json'